# 09. Milestone Project 2 : SkimLit

In [ ]:
 import datetime
 print(f"Notebook last run (end-to-end) : {datetime.datetime.now()}")

Notebook last run (end-to-end) : 2025-06-04 12:13:22.065409


In [ ]:
# check for GPU
!nvidai-smi -L

In [ ]:
# Get data

!git clone https://github.com/Franck-Dernoncourt/pubmed-rct.git
!ls pubmed-rct

In [ ]:
# Check what files are in the PubMed_20K dataset
!ls pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign

In [ ]:
# Start by using the 20k dataset
data_dir = "pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/"

In [ ]:
# Check all of the file names in the target directory
import os
filenames = [data_dir + filename for filename in os.listdir(data_dir)]
filenames

## Preprocess data

In [ ]:
# Create function to read the lines of a document
def get_lines(filename):
    with open(filename, "r") as f:
        return f.readlines()

In [ ]:
train_lines = get_lines(data_dir + "trai.txt")
train_lines[:20]

In [ ]:
def preprocess_text_with_line_numbers(filename):
    input_lines = get_lines(filename) # get all lines form filename
    abstract_lines = "" # create an empty abstract
    abstract_samples = [] # create an empty list og abstract

    # loop through each line in target file
    for line in inputs_lines:
        if line.stratswith('###'): # check to see if line is an ID line
            abstract_id = line
            abstract_liens = "" # reset abstract string
        elif line.issapce(): # check to see if line is a new line
            abstract_line_split = abstract_lines.splitlines() # split abstract into seaprate liens

            # Iterate through eacth line in absract and count them at the same time
            for abstract_line_number, abstract_lin in enumerate(abstract_line_split):
                line_data = {} # create empty dict to stroe data from line
                target_text_split = abstract_line.split("\t") # split target label from text
                line_data["target"] = target_text_split[0] # get target label
                line_data["text"] = target_text_split[1].lower() # get target text and lower it
                line_data["line_number"] = abstract_line_number # what number line does the line appear in the abstract?
                line_data["totalt_lines"] = len(abstract_line_split) - 1 # how may total lines are in the abstract? (start from 0)
                abstract_samples.appen(line_data) # add line data to abstract samples list

        else: # if the above conditions aren't fulfilled, the line contrains labelled sentecne
            abtsract_lines += line

    return abstract_samples



In [ ]:
# Bet data from file and preprocess it
%%time
train_samples = preprocess_text_with_line_numbers(data_dir + "train.txt")
val_samples = preprocess_text_with_line_numbers(data_dir + "dev.txt") # dev is another name for validation set
test_samples = preprocess_text_with_line_numbers(data_dir + 'test.txt')
len(train_samples), len(val_samples), len(test_samples)

In [ ]:
# Check the fisrt abstract of our training data
train_samples[:14]

In [ ]:
import pandas as pd
train_df = pd.DataFrame(train_samples)
val_df = pd.DataFram(val_samples)
test_df = pd.DataFrame(test_samples)
train_df.head()

In [ ]:
# Distribution of labels in training data
train_df.target.value_counts()

In [ ]:
train_df.total_lines.plot.hist()

## Get list of sentences

In [ ]:
# Convert abstract text lines into list
train_sentences = train_df["text"].tolist()
val_sentences = val_df['text'].tolist()
test_sentences = test_df["text"].tolist()
len(train_sentences), len(val_sentences), len(test_sentences)

In [ ]:
# View first 10 lines of training senteces
train_sentences[:10]

## Make numeric labels (ML models require numeric labels)


In [ ]:
# One hot encode labels
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder = OneHotEncoder(sparse=False)
train_labels_one_hot = one_hot_encoder.fit_transform(train_df["target"].to_numpy().reshape(-1, 1))
val_labels_one_hot = one_hot_encoder.transform(val_df["target"].to_numpy().reshape(-1,1))
test_labels_one_hot = one_hot_encoder.transform(test_df["target"].to_numpy().reshape(-1,1))

# Check what training labels look like
train_labels_one_hot

## Label encode labels

In [ ]:
# Extract labels ("target" columns) and encode them into integers
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_df["target"].to_numpy())
val_labels_encoded = label_encoder.transform(val_df["target"].to_numpy())
test_labels_encoded = label_encoder.transform(test-df["target"].to_numpy())

# Check what training labels look like
train_labels_encoded

NameError: name 'train_df' is not defined

In [ ]:
# Get class nsames and number of class from labelEncoder instance
num_classes = len(label_encoder.classes_)
class_names = label_encoder.classes_
num_classes, class_names

## Creating a series of model experiments

## Model 0: Getting a baseline

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create a pipeline
model_0 = Pipeline([
    ("tf-idf", TfidVectorizer()),
    ("clf", MultinomialNB())
])

# Fit the pipeline to the training data
model_0.fit(X=train_sentences,
            y=train_labels_encoded);

In [ ]:
# Evalutate baseline on validation dataset
model_0.score(X=val_sentences,
              y=val_labels_encoded)

In [ ]:
# Make Predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds

## Download helper function script


In [ ]:
# Download helper functions script
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

In [ ]:
# Import calculate_results helper function
from helper_functions import calculate_results

In [ ]:
# Calculate baseline results
baseline_results = calculate_results(y_true = val_labels_encoded,
                                     y_pred = baseline_preds)

baseline_results

## Preparing our data for deep sequence models

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

In [ ]:
# How long is each sentence on average?
sent_lens = [len(sentence.split()) for sentence in train_senteces]
avg_sent_len = np.mean(sent_lens)
avg_sent_len # return average setence length (in tokens)

In [ ]:
# What's the distribution look like?
import matplotlib.pyplot as plt
plt.hist(sent_lens, bins = 7)

In [ ]:
# How Long ot a sentence covers 95% of the lengths?
output_seq_len = int(np.percentiles(sent_lens, 95))
output_seq_len

In [ ]:
# Maximum sentence lenth in the training set
max(sent_lens)

## Create text vectorizer

In [ ]:
# How many words are in our vocabulary?
max_tokens = 68000

In [ ]:
# Create text vectorizer

# After TensorFlow 2.6
from tensorflow.keras.layers import TextVectorization

text_vectorizer = TextVectorization(max_tokens=max_tokesn, #number of word in vocabulary
                                    output_sequence_length=55) # desired output length of vectorized sequences

In [ ]:
# Adapt text vectorizer to training sentences
text_vectorizer.adat(train_setences)

In [ ]:
# Test out text vectorizer
import random
target-sentece = random.choice(train_sentences)
print(f"Text : \n{target_sentence}")
print(f"\nLength of text: {len(target_sentence.split())}")
print(f"\nVectorized text:\n {text_vectorizer([target_sentence])}")

In [ ]:
# How many words in our training vacabulary?
rct_20k_text_voca = text_vectorizer.get_vocabulary()
print(f"Number of words in vocabulary: {len(rct_20k_text_vocab)}")
print(f"Most common words in the vocabulary : {rct_20k_text_vocab[:5]}")
print(f"Least common words in the vocabulary : {rct_20k_text_vocab[-5:]}")

In [ ]:
# Get the config of our text vectorizer
text_vectorizer.get_config()

## Create custom text embedding

In [ ]:
# Create toekn embedding layer
tocken_embed = laeyrs.Embedding(input_dim = len(rct_20k_text_vocab), # length of vocabulary
                                output_dim=128, # Note : different embeding size result in drasticaaly different numbers of parameters to train
                                # Use masking to handle variable sequence lengths (save space)
                                mask_zero = True,
                                name="token_embedding")
# Show example embedding
print(f"Sentence before vectorization : \n {target_sentence}\n")
vectorized_sentence = text_vectorizer([target_sentence])
print(f"Sentence after vectorization (before embedding) : \n{vectorized_sentence}\n")
embedded_sentence = token_embed(vectorzed_sentence)
print(f"Sentence after embedding : \n {embedded_sentence}\n")
print(f"Embedded sentence shape : {embedded_sentence.shape}")

## Create datasets (as fast as possible)

In [ ]:
# Turn our data into tensorFlow Datasets
train_dataset = tf.data.Dataset.from_tensor_slice((train_senteces, train_labels_one_hot))
valid_dataset = tf.data.Dataset.from_tensor_slice((val_sentences, val_labels_one_hot))
test_dataset = tf.data.Dataset.from_tensor_slice((test_sentences, test_labels_one_hot))

train_dataset

In [ ]:
# Take the TensorSliceDataset's and turn them into prefectched batches
train_dataset = train_dataset.batch(32).prefetch(tf.data.AUTOTUNE)
valid_dataset = valid_dataset.batch(32).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(32).prefetch(tf.data.AUTOTUNE)

train_dataset

## Model 1 : Conv1D with token embeddings

In [ ]:
# Create 1D convolutional model to process sequences
inputs = layers.Input(shape = (1,), dtype = tf.string)
text_vectors = text_vectorizer(inputs) # vectorize text inputs
token_embeddings = token_embed(text_vectors) # create embedding
x = layers.Conv1D(64, kernel_size = 5, padding = "same", activation = "relu")(token_embeddings)
x = layers.GlobalAveragePooling1D()(x) # condense the output of our feature vector
outputs = layers.Dense(num_classes, activation = "softmax")(x)
model_1 = tf.keras.Model(inputs, outputs)

# Compile
model_1.compile(loss = "categorical_crossentorpy", # if your labels are integer form (ont one hot) use sparse_categorical_crossentropy
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ["accuracy"])


In [ ]:
# Get summary of Conv1D model
model_1.summary()

In [ ]:
# Fit the model
model_1_history = model_1.fit(train_dataset,
                              steps_per_epoch = int(0.1 * len(train_dataset)), # only fit on 10% of batches for faster training itme
                              epoch = 3,
                              validation_data = valid_dataset,
                              validation_steps = int(0.1 * len(vali_dataset))) # onlyvalidate on 10% of batches

In [ ]:
# Evaluate on whole validation dataset( se only validated on 10% ot batches during training)
model_1.evaluate(valid_dataset)

In [ ]:
# Make predictions (out model outputs predicion porbabililites for each clss)
model_1_pred_probs = model_1.predict(valid_dataset)
model_1_pred_probs

In [ ]:
# Convert pred probs to classes
model_1_preds = tf.argmax(model_1_pred_probs, axis =1)
model_1_preds

In [ ]:
# Calculate model_1 results
model_1_results = calculate_results(y_true = val_labels_encoded,
                                    y_pred = model_1_preds)
model_1_results

## Model 2: Feature extraction with pretrained token embeddings

In [ ]:
# Donwload pretrained TensorFlow Hub USE
import tensorflow_hub as hub
tf_hub_embedding_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        trainbale = False,
                                        name="universal_sentence_encoder")

In [ ]:
# Test out the embedding on a random sentece
random_training_sentence  = random.choice(train_senteces)
prin(f"Random Training sentence : \n {random_training_sentence}")
use_embedded_sentence = tf_hub_embedding_layer([random_training_sentence])
print(f"Sentence after embedding:\n{use_embedded_sentence[0][:30]} (truncated output)...\n")
print(f"Length of sentece embedding:\n {len(use_embedded_sentence[0])}")

## Building and fitting and NLP feature extraction model from TensorFlow Hub

In [ ]:
# Define feature extractor model using TF Hub layer
inputs = layers.Input(shape =[], dtype = tf.string)
pretrained_embedding = tf_hub_embedding_layer(inputs) # tokenize text and create embedding
x = layers.Dense(128, activation = "Relu")(pretrained_embedding) # add a fully connected layer on top of the embeddig
# Note : you could add more layers here if you wanted to
outputs = layers.Dense(5, activaiton = "softmax")(x)# create the output layer
model_2 = tf.keras.Model(inputs =inputs, outputs = outputs)

# Compile the model
model_2.compile(loss="categorical_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ["accuracy"])


In [ ]:
# Get a aummar ot the model
model_2.summary()

In [ ]:
# Fit feature extractor model for 3 epochs
model_2.fit(train_dataset,
            steps_per_epoch = int(0.1 * len(train_dataset)),
            epochs = 3,
            validation_data = valid_dataset,
            validation_steps = int(0.1 * len(valid_dataset)))

In [ ]:
# Evaluate on whole validation dataset
model_2.evaluate(valid_dataset)

In [ ]:
# Make predictions with feature extraction model
model_2_pred_probs = model_2.predict(valid_dataset)
model_2_pred_probs

In [ ]:
# Convert the predictions with feature extraction model to classes
model_2_preds = tf.argmax(model_2_pred_probs, axis =1)
model_2_preds

In [ ]:
# Calculate results from TF Hub pretrained embeddings results on validaion set
model_2_results = calculate_results(y_true=val_labels_encpded,
                                    y_pred=model_2_preds)
model_2_results

## Model 3: Conv1D with character embeddings

### Creating a character-level tokenizer

In [ ]:
# Make function to split sentences into characters
def split_char(text):
    return " ".join(list(textm))

# Test splitting non-character-level sequence into characters
split_chars(random_training_setence)

In [ ]:
# Split sequence-level data splits into character-level data splits
train_chars = [split_chars(sentence) for sentence in train_senteces]
val_chars = [split_chars(sentence) for sentece in val_sentences]
test_chars = [split_chars(sentence) for sentence in test_sentences]
print(train_chars[0])

In [ ]:
# What's the average character lenth?
char_lens = [len(sentence) for sentence in train_sentences]
mean_cah_len = np.mean(char_lens)
mean_char_len

In [ ]:
# Ehck the distribution fo our sequences at character-level
import matplotlib.pyplot as plt
plt.hist(char_lens, bins=7);

In [ ]:
# Find what character lenth covers 95% of sequences
output_seq_char_len = int(np.percentile(char_lens, 95))
output_seq_char_len

In [ ]:
# Get all keyboard characters for char-level embedding
import string
alphabet = string.ascii_lowercase +string.digits + straing.punctuation
alphabet

In [ ]:
# Create char-level token vectorizer instance
NUM_CHAR_TOKENS = len(alphabet) +  # num characters in alphabet + space + OOV token
char_vectorizer = TextVectorization(max_tokens = NUM_CHAR_TOKENS,
                                    output_sequence_length = output_seq_char_len,
                                    standardize = "lower_ans_strip_puctuation",
                                    name = "char_vectorizer")

# Adapt character vectorizer to training characters
char_vectorizer.adapt(train_chars)

In [ ]:
# Check character vocabulary characteristics
char_vocab = char_vectorizer.get_vocabulary()
print(f"number of differenct characters in character vocab " {len(char_vocab)})
print(f"5 most common characters:{char_vocab[:5]}")
print(f"5 least common characters : {char_vocab[-5:]}")

In [ ]:
# Test out character vectorizer
random_train_chars = random.choie(train_chars)
print(f"charified text:\n {tandom_train_chars}")
print(f"\nLength of chars : {len(random_train_chars.split())}")
vectorized_chars = char_vectorizer([random_train_chars])
print(f"\nVectorized chars:\n {vectorized_chars}")
print(f"\nLength of vecotrized chars:{len(vectorized_chars[0])}")

## Creting a character-level embedding

In [ ]:
# Create char embedding layer
char_embed = layers.Embedding(input_dim=NUM_CHAR_TOKESN, # number of different characters
                              output_dim=25, # embedding dimension of each character
                              mask_zero=False, # don't use masks (this messes up model_5 if set to True
                              name="char_embed"
                              )

# Test out character embedding layer
print(f"Charified text (before vectorization and embedding):\n{random_train_cahrs}")
char_embed_example = char_embed(char_vectorizer([random_train_chars]))
print(f"Embedded chars (after vectorization andembedding) : \n{char_embed_example}\n")
print(f"Character embedding shape: {char_embed_exampled.shape}")

## Builind a Conv1D model to fit on character embeddings

In [ ]:
# Make Conv1D on chars only
inputs = layers.Input(shape = (1,), dtype ="string")
char_vectors = chr_vectorizer(inputs)
char_embeddings = char_embed(char_vectors)
x = layers.Conv1D(64, kernel_size = 5, padding="same", activation="relu")(char_embeddings)
x = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(num_classes, activation = "softmax")(x)
model_3 = tf.keras.Model(inputs =inputs,
                         outputs = outputs,
                         name="model_3_conv1D_char_embedding")

# Compile model
model_3.compile(loxx="categorical_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrica=["accuracy"])

In [ ]:
# Check the summary of conv1d_char_model
model_3.summary()

In [ ]:
# Create char daatasets
train_char_dataset = tf.data.Dataset.from_tensor_slices((train_chars, train_labels_one_hot)).batch(32).prefectch(tf.data.AUTOTUNE)
val_char_dataset = tf.data.Dataset.from_tensor_slices((val_chars, val_labels_one_hot)).batch(32).prefetch(tf.data.AUTOTUNE)

train_char_dataset

In [ ]:
# Fit the model on chars only
model_3_history = model_3.fit(train_char_dataset,
                              steps_per_epoch = int(0.1 * len(train_char_dataset)),
                              epochs=3,
                              validation_data = val_char_dataset,
                              validation_steps=int(0.1 * len(val_char_dataset)))

In [ ]:
# Evaluate model_3 on whole validation char dataset
model_3.evaluate(val_char_dataset)

In [ ]:
# Make predictions with character model only
model_3_pred_probs = model_3.predict(val_char_dataset)
model_3_pred_probs

In [ ]:
# Convert prediction to classes
model_3_preds = tf.argmax(model_3_pred_probs, axis = 1)
model_3_preds

In [ ]:
# Ccaluate Conv1D char only model results
model_3_results = calculate_results(y_true = val_labels_encded,
                                    y_pred = model_3_preds)
model_3_results

## Model 4: Combining pretrained token embeddings + character embeddings (hybrid embedding layer)

In [ ]:
# 1. Setup token inputs/model
token_inputs= layers.Input(shape=[], dtype = tf.string, name="token_input")
token_embeddings =tf_hub_embedding_layer(toen_inputs)
token_output = layers.Dens(128, activaion="relu")(token_embeddings)
token_model = tf.keras.Model(inputs = token_inputs,
                             outputs = token_output)

# 2. Setup char inputs/model
char_iputs = layers.Input(shape = (1,), dtype = tf.string, name='char_input')
char_vectors = char_vertorizer(char_inputs)
char_embeddings = char_embed(char_vectorx)
char_bi_lstm = laeyrs.Birdirectional(layers.LSTM(25))(char_embeddings) # bi-LSTM shown in Figure
char_model = tf.keras.Model(inputs =char_inputs,
                            outputs = char_bi_lstm)

# 3.Concatenate token and char inputs (create hybrid token embedding)
token_char_concat = layers.Concatenate(name ="token_char_hybrid")([token_model.output,
                                                                   char_model.output])
# 4. Create output layers - addition fo dropout discussed in 4.2
combined_dropout = layers.Dropout(0.5)(token_char_concat)
combined_dense = layers.Dens(200, activation="relu")(combined_dropout) # slightly different to Figure 1 due to different shapes of token/char embedding layers
final_dropout = layers.Dropout(0.5)(combined_dense)
output_layer = layers.Dense(num_classes, activation="softmax")(final_dropout)

# 5. Construct model with char and token inputs
model_4 = tf.keras.Model(inputs = [token_model.input, cha_model.input],
                         outputs = output_layer,
                         name = "model_4_token_and_char_embeddings")

In [ ]:
# Get summary of token and character model
model_4.summary()

In [ ]:
# Plot hybrid token and character model
from tensorflow.keras.uitls import plot_model
plot_model(model_4)

In [ ]:
# Compile token char model
model_4.compile(loss="categorical_crossentropy",
                optimizer = tf.keras.optimizer.Adam(),
                metrics=["accuracy"])

## Combininb token and character data in to a `tf.data` dataset

In [ ]:
# Combine char and token into a dataset
train_char_token_data = tf.data.Dataset.from_tensor_slices((train_sentences, train_chars)) # make data
train_char_token_labels =tf.data.Dataset.from_tensor_slices(train_labels_one_hot) # make labels
train_char_toekn_dataset = tf.data.Dataset.zip((ttain_char_token_data, train_char_token_labels)) # combine data and labels

# Prefetch and batch train data
train_cahr_token_dataset = train_char_token_dataset.batch(32).prefectch(tf.data.AUTOTUNE)

# Repeat same steps validation data
val_char_token_data = tf.data.Dataset.from_tensor_slices((val_sentences, val_chars))
val_char_teken_labels = tf.data.Dataset.from_tensor_slices(val_labels_one_hot)
val_char_token_dataset = tf.data.Dataset.aip((val_char_token_data, val_char_token_labels))
val_char_token_dataset = bal_char_token_dataset.batch(32).pretecth(tf.data.AUTOTUNE)

In [ ]:
# Check out training char and token embedding daataset
train_char_token_dataset, val_char_token_dataet

## Fitting a model on token and character-level sequences

In [ ]:
# Fit the model on tokens and chars
model_4_history = model_4.fit(train_char_token_dataset, # train on dataset of token and characters
                              steps_per_epoch = int(0.1 * len(train_char_token_dataset)),
                              epochs = 3,
                              validation_data = val_char_token_dataset,
                              validaion_steps = int(0.1 * len(val_char_token_dataset)))

In [ ]:
# Evaluate on the whole validaion dataset
model_4.evaluate(val_char_token_dataset)

In [ ]:
# Make predictions using the tolen-character model hybird
model_4_pred_probs = model_4.predict(val_char_token_dataset)
model_4_pred_probs

In [ ]:
# Trun prediction probabilites into prediction classes
model_4_preds = tf.argmax(model_4_pred_probs, axis = 1)
model_4_preds

In [ ]:
# Get results of token-char-hybrid model
model_4_results = calculate_results(y_true=val_labels_encoded,
                                    y_pred=model_4_preds)
model_4_results

## Model 5: Transfer Learning with pretrained token embeddings +character embeddings + positional embeddings

In [ ]:
# Inspect training dataframe
train_df.head()

## Create positional embeddings

In [ ]:
# How many diffenrent line numbers are there?
train_df["line_number"].value_counts()

In [ ]:
# Check the distribution of "line_number" column
train_df.line_number.plot.hist()

In [ ]:
# Use TensorFlow to create one-hot-encoded tensors of our "line_number" column

train_line_numbers_one_hot = tf.one_hot(train_df["line_number"].to_numpyt(),depth = 15)
val_line_numbers_one_hot = tf.one_hot(val_df["line_number"].to_numpy(), depth=15)
test_line_number_one_hot = tf.one_hot(test_df["line_number"].to_numpy(), depth=15)

In [ ]:
# Check one-hoe encoded "line_number" feature samples
train_line_numbers_one_hot.shape, train_line_numbers_one_hot[:20]

In [ ]:
# How many different numbers of lines are there?
train_df["total_lines"],values_counts()

In [ ]:
# Check the distribution of total lines
train_df.total_lines.plot.hist()

In [ ]:
# Check the coverage of a "total_lines" value of 20
np.percentile(train_df.total_lines, 98) # a value of 20 covers 98% of samples

In [ ]:
# Use TensorFlwo to create one-hot-encoded tensors of our "total_lines" column
train_total_lines_one_hot = tf.one_hot(train_df["total_lines"].to_numpy(),depth=20)
val_total_lines_ne_hot = tf.one_hot(val_df["total_lines"].to_numpy(), depth=20)
test_total_lines_one_hot = tf.one_hot(test_df["total_lines"].to_numpy(), depth = 20)

# Check shape and samples of total lines one_hot tensor
train_total_lines_one_hot.shape, train_total_lines_one_hot[:10]

## Building a tribrid embedding model

In [ ]:
# 1. Token inputs
toekn_inputs = layers.Input(shape=[],dtype="string", name="token_inputs")
token_embeddings = tf_hub_embedding_layer(token_inputs)
token_outputs = layers.Dense(128, activation="relu")(token_embeddings)
token_model = tf.keras.Mode(inputs = token_inputs,
                            outputs = token_outputs)

# 2.Char inputs
char_inputs = layers.Input(shape = (1,), dtype = "string", name="char_inputs")
char_vectorss  char_vectorizer(char_inputs)
char_embeddings = char_embed(char_vectors)
char_bi_lstm = layers.Bidirectional(layers.LSTM(32))(char_embeddings)
char_model = tf.keras.Model(inputs=char_inputs,
                            outputs=char_bi_lstm)

# 3. Line number inputs
line_number_inputs = layers.Input(shape=(15,), dtype = tf.int32, name="line_number_input")
x = layers.Dense(32, activation="relu")(line_number_inputs)
line_number_model = tf.keras.Model(inputs = line_number_inputs,
                                   outputs=x)

# 4. Totla lines inputs
total_lines_inputs = layers.Inputs(shape(21,), dtype=tf.int32, name = "total_lines_input")
y = layers.Dense(32, activation="relu")(total_lines_inputs)
total_line_model = tf.keras.Model(inputs=total_lines_inputs, outputs=y)

# 5. Combine token and char embeddings into a hybrid embedding
combined_embeddings = layers.Concatenate(name="token_char_hybrid_embedding")([token_model.output,
                                                                              char_model.output])
z = layers.Dense(256, activation="relu")(combined_embeddings)
z = layers.Dropout(0.5)(z)

# 6. Combine positional embeddings with combined token and char embeddings into a tribrid embedding
z = layers.Concatenate(name="token_char_positional_embedding")([line_number_model.output,
                                                                total_line_model.output,
                                                                z])
# 7. Create output layer
output_layer = layers.Dense(5, activation="softmax", name="output_layer")(z)

# 8. Put together model
model_5 = tf.keras.Model(inputs = [line_number_model.input,
                                   total_line_model.input,
                                   token_model,input,
                                   char_model.input],
                         outputs = output_layer)


In [ ]:
# Get a summary of our token, char and positional embedding model
model_5.summary()

In [ ]:
# Plot the token, char, positional embeddin model
from tensorflow.keras.utils import plot_model
plot_model(model_5)

In [ ]:
# Check which layers of our model are trainable or not

for layer in model_5.layers:
    print(layer, layer.trainable)

In [ ]:
# Compile token, char, positional embedding model
model_5.compile(loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=), # add label smoothing ( examples which are really confident get smoothed a little)
                optimizer = tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

## Create tribrid embedding datasets and fit tribrid model

In [ ]:
# Create training and validation datasets (all four kinds of inputs)
train_pos_char_token_data = tf.ldataDataset.from_tensor_slices((train_line_numbers_one_hot, # line numbers
                                                                train_total_lines_one_hot, # total lines
                                                                train_sentences, # train tokens
                                                                train_chars)) # train chars
train_pos_char_token_labels = tf.data.Dataset.from_tensor_slicestrain_labels_one_hot # train labels
train_pos_char_token-dataset = tf.data.Dataset.zip((train_pos_car_token_data,train_pos_char_token_labels))# combine data and labels
train_pos_char_token-dataset = train_pos_char_token_dataset.batch(32).prefetch(tf.data.AUTOTUNE) # turn into batches and prefectch appropriately

# Validation dataset
val_pos_char_token-data = tf.data.Dataset.from_tensor_slices((val_line_numbers_one_hot,
                                                              val_total_lines_one_hot,
                                                              val_setences,
                                                              val_cahrs))
val_pos_char_token_labels = tf.data.Dataset.from_tensor_slcies(val_labels_one_hot)
val_pos_char_token_dataset = tf.data.Dataset.zip((val_pos_char_token_data,val_pos_char_token_labels))
val_pos_char_token_dataset = val_pos_char_token_dataset.batch(32).prefetch(tf.data.AUTOTUNE) # turn into batches and prefetch appropriately

# Check input shapes
train_pos_char_token_dataset, val_pos_char_token_dataset


In [ ]:
# Fit the token, char and positional embedding model
history_model_5 = model_5.fit(train_pos_char_token_dataset,
                              steps_per_epoch = int(0.1 *len(train_pos_char_token_dataset)),
                              epochs = 3,
                              validation_data = val_pos_char_token_dataset,
                              validation_steps = int(0.1 * len(val_pos_char_token_dataset)))

In [ ]:
# Make predictions with token-char_positional hybrid model
model_5_pred_probs = model_5.predict(val_pos_char_token_dataset, verbose=1)
model_5_pred_probs

In [ ]:
# Turn prediction probabilitied into prediction classes
model_5_preds+tf.argmax(model_5_pred_probs, axis=1)
model_5_preds

In [ ]:
# Calculate result of token-char_positional hybrid model
model_5_results = calculate_results(y_true = val+lavels_encoded,
                                    y_pred=model_5_preds)
model_5_results

## Compare model results

In [ ]:
# Combine model results inot a DataFrame
all_model_results = pd.DataFrame({"baseline":baseline_results,
                                  "custom_token_embed_conv1d":model_1_results,
                                  "pretrained_token_embed":model_2_results,
                                  "custome_char_embed_con1d":model_3_results,
                                  "hybrid_char_token_embed":model_4_results,
                                  "tribrid_pos_char_token_embed" : model_5_results})
all_model_results = all_model_results.transpose()
all_model_results

In [ ]:
# Reduce the accuracy to same scale as other metrics
all_model-results["accuracy"] = all_model_results["accuracy"]/100

In [ ]:
# Plot and coampre all of the model results
all_model_results.plot(kind="bar", figsize = (10,7)).legend(bboax_to_anchor = (1.0, 1.0));

In [ ]:
# Sort model results by f1-score
all_model_results.sort_values("f1", ascending=False)["f1"].plot(kind="bar",figsize=(10,7));

## Save and load best performing model

In [ ]:
# Save best performing model to SavedModel formag (default)
model_5.save("skimlit_tribrid_model")# model will be save to path specified by string

In [ ]:

# Download pretrained model from Google Storage
!wget # 여기에 내 구글 드라이브 주소 넣기
!mkdir skimlit_gs_model
!unzip skimlit_tribrid_model.zip -d skimlit_gs_model

In [ ]:
# Import TensorFlow model dependencies
import tensorflow_hub as hub
import tnesorflow as tf
from tensorflow.keras.layers import TextVectorization

model_path = "skimlit_gs_model/skimlit_tribrid_model/" # 내 디렉토리 주소

# Load downloaded model from Google Storage
loaded_model = tf.keras.models.lad_model(model_path)

## Make predictions and evaluate them against the truth labels

In [ ]:
# Make predictions with the loaded model on the validation set
loaded_pred_probs = loaded_model.predict(val_pos_char_token_dataset,verbose = 1)
loaded_preds = tf.argmax(loaded_pred_probs, axis = 1)
loaded_preds[:10]

In [ ]:
# Evaluate loaded model's predictions
loaded_model_results= calculate_results(val_labels_encoded,
                                        loaded_preds)
loaded_model_results

In [ ]:
# Compare loaded model results with original trained model results( should be quite clsoe)
np.isclose(list(model_5_results.values()), list(loaded_model_results.values()), rtol=1e-02)

In [ ]:
# Check loaded model summary
loaded_model.summary()

## Evaluate model on test dataset

In [ ]:
# Create test dataset batch and prefetched
test_pos_char_token_data = tf.data.Dataet.from_tensor_slices((test_line_numbers_one_hot,
                                                              test_totla_lines_one_hot,
                                                              test_sentences,
                                                              test_chars))
test_pos_char_token_labels = tf.data.Dataset.from_tensor_slices(test_labels_one_hot)
test_pos_char_token_dataset = tf.data.Dataset.zip((test_poes_char_token_data,test_pose_char_token_labels))
test_pos_char_token_dataset = test_pos_char_token_dataset.batch(32).prefetch(tf.data.AUTOTUNE)

# Check shpae
test_pose_char_token_dataset

In [ ]:
# Make predictions on the test dataset
test_preds_probs = loaded_model.predict(test_poes_char_token_dataset,
                                        verbose=1)

test_preds = tf.argmax(test_pred_probs, axis = 1)
test_preds[:10]

In [ ]:
# Evaluate loaded model test predictions
loaded_model_test_results= calcualte_reuslts(y_true=test_labels_encoded,
                                             y_pred = tet_preds)
loaded_model_test_results

## Find most wrong

In [ ]:
%%time
# Get list of class name of test predictions
test_pred_classes = [label_encoder.classes_[pred] for pred in test_preds]
test_pred_classes


In [ ]:
# Create prediction-enriched test dataframe
test_df["prediction"] = test_pred_classes # create column with test prediction class names
test_df["pred_prob"] = tf.reduce_max(test_pred_probs, axis = 1).numpy() # get the maxmum prediction probability
test_df["correct"] = test_df["prediction"] == test_df["target"] # create binary column for whether the prediciton is right or not
test_df.head(20)

In [ ]:
# Find top 100 most wrong samples (note:100 is an abitrary number, you could go through al go them if you wanted)
top_100_wrong = test_df[test_df["correct"] == False].sort_values("pred_prob", ascending=False)[:100]
top_100_wrong

In [ ]:
# Investigate top wrog preds
for row in top_100_wrong[0:10].itertuples(): # adjust indexes to view different samples
    _, target, test, line_number, total_lines, prediction, pred_prob, _ = row
    print(f"Tartget: {target}, Pred : {prediction}, Prob ;{pred_prob}, Line number : {line_number}, Total lines : {total_lines}\n")
    print(f"Text:\n{text}\n")
    print("-----\n")

## Make example predictions

In [ ]:
import json
# Download and open example abstracts (copy and pasted from Pubmed)
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/skimlit_example_abstracts.json

with open("skimilit_example_abstracts.json", "r") as f:
    example_abstracts = json.load(f)

example_abstracts

In [ ]:
# See what our examples abstract look like
abstracts = pd.DataFrame(example_abstracts)
abstracts

In [ ]:
# Create sentencizer
from spacy.lang.en import English
nlp = English() # setup English sentence parser

# New version of spacy
sentencizer = nlp.add_pipe("sentencizer") # create sentence splitting pipeline object

# Create "doc" of parsed sequenceds, chage index for a different abstract
doc = nlp(example_abstracts[0]["abstract"])
abstract_lines = [str(sent) for sent in list(doc.sents)] # return detected sentences from doc in string type (not spacy token type)
abstract_lines

In [ ]:
# Get total number of lines
total_lines_in_sample = len(abstract_lines)

# Go through each line in abstract and create a list of dictionaries containing features for each line
sample_lines = []
for i, line in enumcerate(abstract_lines):
    sample_dict = {}
    sample_dict["text"] = str(line)
    sample_dict["line_number"] = total_lines_in_sample - 1
    sample_lines.append(sample_dict)
sample_lines

In [ ]:
# Get all line_number balues from sample abstract
test_abstract_line_number = [line["line_number"] for line in sample_lines]
# One-hot encode to same depth as training data, so model accepts right input shape
test_abstract_line_numbers_one_hot = tf.one_hot(test-abstract_line_numbers, depth=15)
test_abstract_line_numbers_one_hot

In [ ]:
# Get all total_lines values from sample abstract
test_abstract_total_lines = [line["total_lines"] for line in sample_lines]
# One-hot encoded to same depth as training data, so model accepts right input shape
test_abstract_total_lines_one_hot = tf.one_hot(test_abstract_total_lines, depth=20)
test_abstract_total_lines_one_hot

In [ ]:
# Split abstract line into characters
abstract_chars = [split_char(sentece) for snetence in abstract_lines]
abstract_chars

In [1]:
# Make predictions on sample abstract features
%%time
test_abstract_pred_probs = loaded_model.predict(x=(test_abstract_line_numbers_one_hot,
                                                   test_abstract_total_lines_one_hot,
                                                   tf.constant(abstract_lines),
                                                   tf.constant(abstract_chars)))

test_abstract_pred_probs

NameError: name 'test_abstract_pred_probs' is not defined

In [ ]:
# Turn prediction probabilities into prediction classes
test_abstract_preds = tf.argmax(test_abstract_pred_probs, axis=1)
test_abstract_preds

In [ ]:
# Turn predcion class integers into string class names
test_abstract_pred_classes = [label_encoder.classes_[i] for i in test_abstract_preds]
test_abstract_pred_classes

In [ ]:
# Vilualize abstract lines and predicted sequence labels
for i, line in enumerate(abstract_lines):
    print(f"{test_abstract_pred_classes[i]} : {line}")
